In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import pandas as pd

# 绘制年际相关系数、均方根误差
2021.12.21

因为前处理部分太长，pre 和 t2m 在各自的脚本中绘制

TCCs 和 RMSEs分开绘制并放置

使用了MPAS-92-25KM 实验的绘制办法；TCCs计算了t检验的P值

## 数据读入

目标处理：1991-2015 daily precipitation

In [ ]:
path_in = "/raid61/LS4P-TPEMIP/TEMP_DATA/t2m_mask/"
ds_in = {}
dir_obs_in ="/raid61/LS4P-TPEMIP/REFERENCE/CN05.1/"
ds_in['obs'] = xr.open_dataset(dir_obs_in + "sel_CN05.1_Tm_1961_2018_daily_025x025.nc").rename({"tm":"t2m"})
# ds_in[''] = xr.open_dataset(path_in + "")
ds_in['CWRF'] = xr.open_dataset(path_in + "CWRF_t2m.nc") - 273.15 # 缺少逐日数据，只有多种尺度的平均态数据
ds_in['JAMSTECWRF'] = xr.open_dataset(path_in + "JAMSTECWRF_t2m.nc") - 273.15# 缺少4月
ds_in['NJU_LS4P_B'] = xr.open_dataset(path_in + "NJU_LS4P_B_t2m.nc") - 273.15
ds_in['NJU_LS4P_C'] = xr.open_dataset(path_in + "NJU_LS4P_C_t2m.nc") - 273.15
ds_in['NJU_Output-E'] = xr.open_dataset(path_in + "NJU_Output-E_t2m.nc") - 273.15 # 缺少2015年
ds_in['SYS'] = xr.open_dataset(path_in + "SYS_t2m.nc") - 273.15
ds_in['Yang-ZhouX'] = xr.open_dataset(path_in + "Yang-ZhouX_t2m.nc") - 273.15
ds_in['YSU'] = xr.open_dataset(path_in + "YSU_t2m.nc") - 273.15

In [ ]:
# 稍微处理一下变量名，用于后续的处理
for single_mod in ds_in:
    print(single_mod)
    list_vars = list(ds_in[single_mod].variables)
    print(list_vars)
#     print(ds_in[single_mod].coords)
    if ('Time' in list_vars):
        print('----- rename Time>>>time')
        ds_in[single_mod] = ds_in[single_mod].rename({"Time":"time"})
#     if ('pr' not in list_vars):
#         print('------ rename ???pre???>>>pr')
#         ds_in[single_mod] = ds_in[single_mod].rename({"dailypre":"pr"})
#     if (ds_in[single_mod].time.shape[0] !=3825):
#         print("**********time error in " + single_mod + " !" + " time length:" + str(ds_in[single_mod].time.shape[0]) )

## 计算部分

### 整理年际时间序列

In [ ]:
# 拆分4-5月以及6-8月
var_selmonth = {}
var_selmonth['am'] = {}
var_selmonth['jja'] = {}

for mod_name in ds_in:
    # select month
    time_idx_am = ds_in[mod_name].time.dt.month.isin([4,5])
    time_idx_jja = ds_in[mod_name].time.dt.month.isin([6,7,8])
    # put into dict
    var_selmonth['am'][mod_name]  = ds_in[mod_name]['t2m'].isel(time = time_idx_am)
    var_selmonth['jja'][mod_name] = ds_in[mod_name]['t2m'].isel(time = time_idx_jja)
    
# 获取年际时间序列 var_interannual{} dict
var_interannual = {}
time_for_groupby = {}

for iseason in ['am', 'jja']:
    var_interannual[iseason] = {}
    for mod_name in ds_in:
        time_for_groupby['am'] = var_selmonth['am'][mod_name].time.dt.year
        time_for_groupby['jja'] = var_selmonth['jja'][mod_name].time.dt.year
        var_interannual[iseason][mod_name] = var_selmonth[iseason][mod_name].groupby(time_for_groupby[iseason]).mean(dim = 'time')

### 计算年际相关

In [ ]:
def scipy_count_corr_2d(a,b):
    '计算两个3D序列在时间维上的相关性，使用scipy逐个格点的计算，获取pvalues,输入数组a,b 按照 time x lat x lon的方式进行排列'
    '如果第一个时次出现了np.nan ,那么就对这个格点赋值为nan'
    import numpy as np
    from scipy import stats

    dim1 = a.shape[1]
    dim2 = a.shape[2]
    pvalues = np.empty(shape = a.shape[1:])
    corrvalues = np.empty(shape = a.shape[1:])
    # np.corrcoef?
    for ilat in range(0, dim1):
        for ilon in range(0, dim2):
            if ( (np.isnan(a[0,ilat,ilon])) | (np.isnan(b[0,ilat,ilon])) ):
                corrvalues[ilat, ilon], pvalues[ilat, ilon] = np.nan, np.nan
                continue    
            corrvalues[ilat, ilon], pvalues[ilat, ilon] \
                = stats.pearsonr(a[:,ilat,ilon], b[:,ilat,ilon])
    return [corrvalues, pvalues]

In [ ]:
corr_interannual = {}

for iseason in ['am', 'jja']:
    corr_interannual[iseason] = {}
    for mod_name in ds_in:
        if (mod_name == 'CWRF'): # CWRF暂时只有平均态，所以不参与到计算之中
            continue
        corr_interannual[iseason][mod_name]  = {}
        corr_interannual[iseason][mod_name]['corr']    = \
            xr.corr(var_interannual[iseason]['obs'], var_interannual[iseason][mod_name], dim= 'year')
        corr_scipy, pvalues_scipy = \
            scipy_count_corr_2d(var_interannual[iseason]['obs'].values, var_interannual[iseason][mod_name].values)
        corr_interannual[iseason][mod_name]['pvalues'] = xr.DataArray(pvalues_scipy, coords = \
            corr_interannual[iseason][mod_name]['corr'].coords, name = 'pvalues')

### 计算年际均方根误差

In [ ]:
rmse_interannual = {}

for iseason in ['am', 'jja']:
    rmse_interannual[iseason] = {}
    for mod_name in ds_in:
        if (mod_name == "CWRF"):
            continue
        nyears = var_interannual[iseason][mod_name].year.shape[0]
        rmse_temp = np.sqrt( ( (var_interannual[iseason][mod_name].values - var_interannual[iseason]['obs'].values)**2).sum(axis = 0) / nyears)
        rmse_interannual[iseason][mod_name] = xr.DataArray(rmse_temp, coords = \
            corr_interannual[iseason][mod_name]['corr'].coords, name = 'rmse')

## 绘图部分

In [ ]:
# 国内政区图的绘制
# Load the border data, CN-border-La.dat is download from
# https://gmt-china.org/data/CN-border-La.dat
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches
import cmaps
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

In [ ]:
def border_plot(axs):
    """
    进行行政区划的绘制，通过shapefilereader绘制存档的shp文件，需要传入axs，并逐个绘制
    比较消耗时间，调整完毕后最后添加边界的绘制
    """
    #----- 添加海洋以及行政区划 -----
    ##---- 直接绘图，从边界文件添加
    turn_on_border = True
    if(turn_on_border):
        ##---- 使用shp文件添加
            ## shapefile数据下载的位置：
        coast_shapefile = "/home/yycheng/playground/shapefiles/gshhg-shp/GSHHS_shp/l/GSHHS_l_L1.shp"
        ninelines_shapefile     = "/home/yycheng/playground/shapefiles/SouthSea/" + "nine_lines.shp"
        ## 来源： 沛沛的诸省 + 诸岛
    #     bou24p_shapefile     = "/m2data2/yycheng/data_stage/CN-border/peipeihelp/" + "bou2_4p.shp"
        ## 来源： https://www.resdc.cn/data.aspx?DATAID=200
        province_shapefile     = "/home/yycheng/playground/shapefiles/CN-sheng/" + "change_proj_CN-sheng-A.shp"

        for ax in axs:
            coast = shpreader.Reader(coast_shapefile).geometries()
            # world     = shpreader.Reader(world_border_shapefile).geometries()
            # river     = shpreader.Reader(river_border_shapefile).geometries()
    #         river     = shpreader.Reader(river_border_shapefile, encoding = 'gbk')
            # bou24p    = shpreader.Reader(bou24p_shapefile).geometries()
            ninelines = shpreader.Reader(ninelines_shapefile).geometries()
            province  = shpreader.Reader(province_shapefile).geometries()
            ax.add_geometries(coast, ccrs.PlateCarree(), facecolor="none", edgecolor = 'k', linewidth=0.4, zorder=1 )
            # ax.add_geometries(world, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.4, zorder=1)
            # ax.add_geometries(bou24p, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 沛沛map
            ax.add_geometries(province, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1) # 地资所
            ax.add_geometries(ninelines, ccrs.PlateCarree(), facecolor='none', edgecolor='k', linewidth=0.6, zorder=1)
            # ax.add_geometries(river, ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.4, zorder=1)
            # 绘制部分的shapefile
    #         for region in river.records():
    #             if (region.attributes['NAME'] in ['黄河','长江']):
    #                 # print("----- draw river! -----")
    #                 # 此处需要使用 [] 让region.geometry可以迭代
    #                 ax.add_geometries([region.geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='b', linewidth=0.6, zorder=1)

### 绘制年际相关

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import cmaps
# ----- get filter vars coords-----

lon = corr_interannual['am']['obs']['corr'].lon.values
lat = corr_interannual['am']['obs']['corr'].lat.values
models_list = ['JAMSTECWRF', 'NJU_LS4P_B', 'NJU_LS4P_C', 'NJU_Output-E', 'SYS', 'Yang-ZhouX', 'YSU']

#----- create plot -----
shape_array = [[1,2,3,4,],[5,6,7,0],[8,9,10,11],[12,13,14, 0]]
fig, axs = plot.subplots(shape_array, proj=('cyl'))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- colorbar ticks 统一设置 -----

# cmap_corr = cmaps.ncl_default
cmap_corr = cmaps.cmp_flux
cmap_rmse = cmaps.MPL_gist_ncar
corr_ticks = np.concatenate( [ [0,0.1,0.2] , plot.arange(.3,1, 0.025) ] , axis = 0)
# rmse_ticks = np.linspace(0, 25, 26)
# rmse_ticks = np.concatenate((np.linspace(0,10,21), [12,14,16,18,20,25]), axis=0)

# 绘制borders
border_plot(axs)

for season_ind, season_name in enumerate(['am','jja']):
    for mod_ind, mod_name in enumerate(models_list):
        # corr
        m_corr  = axs[season_ind * 7 +  mod_ind ].contourf(lon, lat, corr_interannual[season_name][mod_name]['corr'].values,\
        levels=corr_ticks,cmap=cmap_corr)
        # 绘制散点
        # 需要将原本密集的格点散点减少 (::4) 然后找到减少后的经纬度lon[::4] 进行散点的绘制
        scatter_test = np.argwhere((corr_interannual[season_name][mod_name]['pvalues'][::4,::4]<=0.01).values)
        axs[season_ind * 7 +  mod_ind].scatter(lon[::4][scatter_test[:,1]],lat[::4][scatter_test[:,0]], s=0.15, color = 'k', marker='o')
        axs[season_ind * 7 + mod_ind].format(ltitle = season_name.upper(), title = mod_name)
#         # rmse
#         m_rmse  = axs[plot_ind + 7].contourf(lon, lat, rmse_interannual[season_name][mod_name].values,\
#         levels=rmse_ticks,cmap=cmap_rmse)

#----- add color bar-----

fig.colorbar(m_corr, loc='b', width=0.1,cols = (1,4),
ticklabelsize=5,ticks=corr_ticks, title='corrleation')

# fig.colorbar(m_rmse, loc='b', width=0.1,cols = (3,4),
# ticklabelsize=5,ticks=rmse_ticks, title='RMSE')

# ---------format作为整个proplot集中对属性进行修改的method---------
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = False,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
#-----GEO axis-----
lonlim=(70, 140), latlim=(15, 55),
labels = False, # 后续自己手动添加带线条的
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,55,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
longrid  = True,
latgrid  = True,
suptitle="interannual TCCs for the surface air temperature",
)

for ax_ind in axs:
    ax_ind.set_xticks( plot.arange(70,140,10) ,crs=ccrs.PlateCarree()) # set longitude indicators
    ax_ind.set_yticks( plot.arange(15,55,10) , crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True,number_format='.0f')#,degree_symbol='')
    lat_formatter = LatitudeFormatter(number_format='.0f')#,degree_symbol='')
    ax_ind.xaxis.set_major_formatter(lon_formatter)
    ax_ind.yaxis.set_major_formatter(lat_formatter)
    
    ax_ind.tick_params(which='minor',direction='inout',length=4,width=1., top = False, right = False, labelsize = 7.)
    ax_ind.tick_params(which='major',direction='out', length=6, width=1.5 , colors='k', top = False, right = False, labelsize = 7.)
    

#----- save figure -----
fig.patch.set_facecolor('white')
fig.savefig('./output_pic/t2m_interannualTCCs_91-15_12.21.png', dpi=600, facecolor= "white")

### 绘制年际均方根误差

In [ ]:
# import proplot as plot
from matplotlib import pyplot as plt
import proplot as plot
import cmaps
# ----- get filter vars coords-----

lon = corr_interannual['am']['obs']['corr'].lon.values
lat = corr_interannual['am']['obs']['corr'].lat.values
models_list = ['JAMSTECWRF', 'NJU_LS4P_B', 'NJU_LS4P_C', 'NJU_Output-E', 'SYS', 'Yang-ZhouX', 'YSU']

#----- create plot -----
shape_array = [[1,2,3,4,],[5,6,7,0],[8,9,10,11],[12,13,14, 0]]
fig, axs = plot.subplots(shape_array, proj=('cyl'))
m_contour_list = [] # 用于保存contour设置，后续设置colorbar使用

#----- colorbar ticks 统一设置 -----

# cmap_corr = cmaps.ncl_default
# cmap_corr = cmaps.cmp_flux
cmap_rmse = cmaps.MPL_s3pcpn_l
# corr_ticks = plot.arange(-0.3,1, 0.05)
# rmse_ticks = plot.arange(0, 5, 0.5)
rmse_ticks = np.concatenate([plot.arange(0, 8, 0.25), [9, 10, 12]] , axis = 0)
# rmse_ticks = np.concatenate((np.linspace(0,10,21), [12,14,16,18,20,25]), axis=0)

# ------ 绘制borders ------
border_plot(axs)

for season_ind, season_name in enumerate(['am','jja']):
    for mod_ind, mod_name in enumerate(models_list):
        # rmse
        m_rmse  = axs[mod_ind + 7 * season_ind].contourf(lon, lat, rmse_interannual[season_name][mod_name].values,\
        levels=rmse_ticks,cmap=cmap_rmse)
        axs[season_ind * 7 + mod_ind].format(ltitle = season_name.upper(), title = mod_name)

#----- add color bar-----

fig.colorbar(m_rmse, loc='b', width=0.1,cols = (1,4),
ticklabelsize=5,ticks=rmse_ticks, title='RMSEs')

# fig.colorbar(m_rmse, loc='b', width=0.1,cols = (3,4),
# ticklabelsize=5,ticks=rmse_ticks, title='RMSE')

# ---------format作为整个proplot集中对属性进行修改的method---------
axs.format(
abc=True,
abcloc = 'ul',
#----- 地图底图设置 -----
# reso = 'x-hi',
reso = 'med',
# coast = False,
coast = False,
coastlinewidth = 0.4,
borders = False,
lakes = False,
land  = False,
ocean = False,
#-----GEO axis-----
lonlim=(70, 140), latlim=(15, 55),
labels = False, # 后续自己手动添加带线条的
gridlabelsize = 5,
gridminor = True,
lonlocator = np.arange(70,150,10),
latlocator = np.arange(15,55,10),
lonminorlocator = np.arange(70,140+2,2),
latminorlocator = np.arange(15,55+2,2),
longrid  = True,
latgrid  = True,
suptitle="interannual RMSEs for the surface air temperature",
)

for ax_ind in axs:
    ax_ind.set_xticks( plot.arange(70,140,10) ,crs=ccrs.PlateCarree()) # set longitude indicators
    ax_ind.set_yticks( plot.arange(15,55,10) , crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True,number_format='.0f')#,degree_symbol='')
    lat_formatter = LatitudeFormatter(number_format='.0f')#,degree_symbol='')
    ax_ind.xaxis.set_major_formatter(lon_formatter)
    ax_ind.yaxis.set_major_formatter(lat_formatter)
    
    ax_ind.tick_params(which='minor',direction='inout',length=4,width=1., top = False, right = False, labelsize = 7.)
    ax_ind.tick_params(which='major',direction='out', length=6, width=1.5 , colors='k', top = False, right = False, labelsize = 7.)
    

#----- save figure -----
fig.patch.set_facecolor('white')
fig.savefig('./output_pic/t2m_interannualRMSEs_91-15_12.21.png', dpi=600, facecolor= "white")